### Import Module

In [1]:
import pandas as pd
import numpy as np

### Prepare columns of interest and specify their types

In [2]:
# Init constant var to store desired cols and their types
DTYPE_DICT = {
    "user_id": pd.Int64Dtype(),
    "status": pd.StringDtype(),
    "timestamp": pd.Int64Dtype(),
    "data.user_data.meta.is_private": pd.BooleanDtype(),
    "data.user_data.meta.is_verified": pd.BooleanDtype(),
    "data.user_data.meta.is_business_account": pd.BooleanDtype(),
    "data.user_data.meta.is_professional_account": pd.BooleanDtype(),
    "data.user_data.meta.has_reel": pd.BooleanDtype(),
    "data.user_data.meta.total_posts_count": pd.Int64Dtype(),
    "data.user_data.meta.followers_count": pd.Int64Dtype(),
    "data.user_data.meta.followings_count": pd.Int64Dtype(),
}

# Iteratively get the metrics for posts
post_column_pattern = "data.user_data.post.{}.{}"
post_metrics = {
    "is_video": pd.BooleanDtype(),
    "video_view_count": pd.Int64Dtype(),
    "comments_count": pd.Int64Dtype(),
    "comments_disabled": pd.BooleanDtype(),
    "taken_at_timestamp": pd.Int64Dtype(),
    "likes_count": pd.Int64Dtype(),
    "is_pinned": pd.BooleanDtype(),
}

# Populate DTYPE_DICT with desired metrics for 12 posts
for i in range(1, 13):
    for metric, dtype in post_metrics.items():
        column_name = post_column_pattern.format(i, metric)
        DTYPE_DICT[column_name] = dtype

# Iteratively get the metrics for reels
reel_column_pattern = "data.user_data.reel.{}.{}"
reel_metrics = {
    "has_audio": pd.BooleanDtype(),
    "video_view_count": pd.Int64Dtype(),
    "comments_count": pd.Int64Dtype(),
    "comments_disabled": pd.BooleanDtype(),
    "taken_at_timestamp": pd.Int64Dtype(),
    "likes_count": pd.Int64Dtype(),
    "video_duration": pd.Float64Dtype(),
}
        
# Populate DTYPE_DICT with desired metrics for 36 reels
for i in range(1, 37):
    for metric, dtype in reel_metrics.items():
        column_name = reel_column_pattern.format(i, metric)
        DTYPE_DICT[column_name] = dtype

# Get a list of desired columns (for `read_csv()`)
columns_to_read = list(DTYPE_DICT.keys())

### Read Data

In [3]:
data = pd.read_csv('.././data/input/instagram_subset.csv',
                  encoding='utf-8',
                  index_col=False,
                  usecols=columns_to_read,
                  dtype=DTYPE_DICT,)

Filter IG account with status success, and has reels. No private accounts

In [4]:
data = data.loc[(data['status'] == 'successful') & (data['data.user_data.meta.has_reel'] == True) & (data['data.user_data.meta.is_private'] == False)]

#filter out the post data, leave only reels and useful columns
data = data.loc[:, ~data.columns.str.contains('data.user_data.post')]
data = data.drop(['status', 'timestamp', 'data.user_data.meta.is_private', 'data.user_data.meta.is_verified', 'data.user_data.meta.is_business_account',	'data.user_data.meta.is_professional_account',	'data.user_data.meta.has_reel',	'data.user_data.meta.total_posts_count'
], axis=1)


Melt the dataframe for each row to contain each reels data

In [5]:
melted = data.melt(id_vars=['user_id', 'data.user_data.meta.followers_count', 'data.user_data.meta.followings_count'], 
                 var_name='variable', 
                 value_name='value')
melted[['reels no.', 'field']] = melted['variable'].str.extract(r'reel\.(\d+)\.(.+)')
melted.sort_values(['user_id'])

stacked = melted.pivot(index=['user_id', 'reels no.', 'data.user_data.meta.followers_count', 'data.user_data.meta.followings_count'], columns='field', values='value').reset_index()
cleaned_data = stacked.rename(columns={'data.user_data.meta.followers_count': 'followers', 'data.user_data.meta.followings_count':'followings'})
cleaned_data

field,user_id,reels no.,followers,followings,comments_count,comments_disabled,has_audio,likes_count,taken_at_timestamp,video_duration,video_view_count
0,1018797,1,4327,1309,5,False,True,24,1720035999,6.238,518
1,1018797,10,4327,1309,1,False,True,13,1687539779,50.033,107
2,1018797,11,4327,1309,0,False,True,9,1686758588,70.033,123
3,1018797,12,4327,1309,0,False,True,9,1683657655,54.066,146
4,1018797,13,4327,1309,0,False,True,8,1680276398,46.0,93
...,...,...,...,...,...,...,...,...,...,...,...
2659,1418637,5,1533,1310,1,False,True,3,1715901334,79.566,1676
2660,1418637,6,1533,1310,0,False,True,3,1714751625,17.437,586
2661,1418637,7,1533,1310,1,False,True,3,1714172546,43.3,1310
2662,1418637,8,1533,1310,0,False,True,42,1712877061,56.473,1812
